# AWS IoT Greengrass Workshop

[Requirements for Greengrass on an EC2 instance](https://docs.aws.amazon.com/greengrass/latest/developerguide/module1.html#setup-filter.ec2)

In [1]:
import boto3
import sys
import os
import json
import base64
import project_path # path to helper methods

from lib import workshop
from botocore.exceptions import ClientError
project_name = 'iot-greengrass-workshop'

ec2_client = boto3.client('ec2')
ec2 = boto3.resource('ec2')

gg = boto3.client('greengrass')
iot = boto3.client('iot')

session = boto3.session.Session()
region = session.region_name

## Create VPC for workshop greengrass EC2 device

In order to simulate a Greengrass device on an EC2 instance we will create a new VPC with a public subnet by running the code below. As you can see to make a subnet public an Internet Gateway is attached to the VPC and a routing table is created with and entry to route all traffic at `0.0.0.0/0` to the Internet gateway. We will store the VPC and Subnet Id's to be used later in the notebook.

In [2]:
vpc, subnet = workshop.create_and_configure_vpc()
vpc_id = vpc.id
subnet_id = subnet.id
print(vpc_id)
print(subnet_id)

vpc-0b15bdb93cec1d509
subnet-0972e999d5ce03dad


### Create the EC2 Keypair

A new EC2 Keypair will be created to allow us to SSH into the EC2 instance. We could have also used the Systems Manager Session Manager here as well.

In [ ]:
!mkdir iot
!mkdir iot/bp
!mkdir iot/hr
!mkdir iot/ht

In [ ]:
try:
    response = ec2_client.describe_key_pairs(
    KeyNames=[
        project_name,
    ],
)
except ClientError as e:
    if e.response['Error']['Code'] == 'InvalidKeyPair.NotFound':
        print ('Creating keypair: %s' % project_name)
        # Create an SSH key to use when logging into instances.
        outfile = open('iot/'+project_name + '.pem','w')
        key_pair = ec2.create_key_pair(KeyName=project_name)
        KeyPairOut = str(key_pair.key_material)
        outfile.write(KeyPairOut)
        outfile.close()
        os.chmod('iot/'+project_name + '.pem', 400)
    else:
        print ('Keypair: %s already exists' % project_name)

### Create Security Group for EC2 instance

The security group will open ports `22` and `8883` respectively for SSH and MQTT access.

In [ ]:
sec_group = ec2_client.create_security_group(
    Description='Security Group for EC2 instance acting as IoT Greengrass device',
    GroupName=project_name+'-sg',
    VpcId=vpc_id
)

sec_group_id=sec_group["GroupId"]
print(sec_group_id)

In [ ]:
data = ec2_client.authorize_security_group_ingress(
    GroupId=sec_group_id,
    IpPermissions=[
        {'IpProtocol': 'tcp',
         'FromPort': 22,
         'ToPort': 22,
         'IpRanges': [
            {
                'CidrIp': '0.0.0.0/0',
                'Description': 'SSH access'
            },
          ]
        },
        {'IpProtocol': 'tcp',
         'FromPort': 8883,
         'ToPort': 8883,
         'IpRanges': [
            {
                'CidrIp': '0.0.0.0/0',
                'Description': 'MQTT access'
            },
          ]
        }
    ]
)

print(data)

### Get latest Amazon Linux AMI in the region

We will lookup the latest AMI version of the Amazon Linux OS to be used for the EC2 instance.

In [ ]:
ami = workshop.get_latest_amazon_linux()
print(ami)

### Create the EC2 instance to act as the edge device using Greengrass

The UserData section of the EC2 instance launch includes everything needed to configure and install Greengrass on the EC2 instance. View the UserData below to get an understanding of what's involved to configure Greengrass on devices. [Greengrass Core downloads](https://docs.aws.amazon.com/greengrass/latest/developerguide/what-is-gg.html#gg-core-download-tab) link provides the available devices and OS's available.

Before running the EC2 create instance cell change the `REGION=<CHOOSE REGION>` to the region you are running in. i.e. `REGION=us-west-2`

In [ ]:
!cat userdata.sh

In [ ]:
fh=open("userdata.sh")
userdata=fh.read()
fh.close()

userdataencode = base64.b64encode(userdata.encode()).decode("ascii")

response = ec2_client.run_instances(
    BlockDeviceMappings=[
        {
            'DeviceName': '/dev/xvda',
            'Ebs': {

                'DeleteOnTermination': True,
                'VolumeSize': 10
            },
        },
    ],
    ImageId=ami,
    InstanceType='t3.micro',
    MaxCount=1,
    MinCount=1,
    KeyName=project_name,
    Monitoring={
        'Enabled': True
    },
    NetworkInterfaces=[{
        "DeviceIndex": 0,
        "SubnetId": subnet_id,
        "Groups": [
            sec_group_id
        ],
        "AssociatePublicIpAddress": True
    }],
    UserData=userdataencode,
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'iot_greengrass_device'
                },
            ]
        },
    ]
)

instance_id = response['Instances'][0]['InstanceId']

waiter = ec2_client.get_waiter('instance_status_ok')
waiter.wait(
    InstanceIds=[
        instance_id,
    ])

print(instance_id+' created successfully')

### Get the Public IP Address of the EC2 instance to ssh into.

In [ ]:
response = ec2_client.describe_instances(
    InstanceIds=[
        instance_id,
    ])

public_ip = response['Reservations'][0]['Instances'][0]['PublicIpAddress']
print(public_ip)

print('ssh -i '+project_name+'.pem ec2-user@'+public_ip)

To SSH into the EC2 instance you can open a terminal from the `+` or `New` notebook menu and select Terminal. Once you have shell access you can `cd ~/SageMaker/iot-greengrass-workshop/notebooks/` and then run the ssh command from the output above

## Setup the IoT sensors and Greengrass Core device in AWS

### Create Thing Type

In [ ]:
thing_type = 'healthtracker'

response = iot.create_thing_type(
    thingTypeName=thing_type
)

### Create the core IoT device

In [ ]:
bp_thing = iot.create_thing(thingName='igw_bp_sensor', thingTypeName=thing_type)

hr_thing = iot.create_thing(thingName='igw_hr_sensor', thingTypeName=thing_type)

ht_group_thing = iot.create_thing(thingName='ht_group_thing', thingTypeName=thing_type)

### Create the Keys and Certificate required for the device

In [ ]:
bp_keys_cert = iot.create_keys_and_certificate(setAsActive=True)

hr_keys_cert = iot.create_keys_and_certificate(setAsActive=True)

ht_keys_cert = iot.create_keys_and_certificate(setAsActive=True)

In [ ]:
print(bp_keys_cert)
print(hr_keys_cert)
print(ht_keys_cert)

### Attach the principal to each device

In [ ]:
iot.attach_thing_principal(thingName=bp_thing['thingName'], principal=bp_keys_cert['certificateArn'])

iot.attach_thing_principal(thingName=hr_thing['thingName'], principal=hr_keys_cert['certificateArn'])

iot.attach_thing_principal(thingName=ht_group_thing['thingName'], principal=ht_keys_cert['certificateArn'])

### Create the IoT Policy for the Greengrass device

In [ ]:
core_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "iot:Publish",
                "iot:Subscribe",
                "iot:Connect",
                "iot:Receive",
                "iot:GetThingShadow",
                "iot:DeleteThingShadow",
                "iot:UpdateThingShadow"
            ],
            "Resource": ["arn:aws:iot:" + boto3.session.Session().region_name + ":*:*"]
        },
        {
            "Effect": "Allow",
            "Action": [
                "greengrass:AssumeRoleForGroup",
                "greengrass:CreateCertificate",
                "greengrass:GetConnectivityInfo",
                "greengrass:GetDeployment",
                "greengrass:GetDeploymentArtifacts",
                "greengrass:UpdateConnectivityInfo",
                "greengrass:UpdateCoreDeploymentStatus"
            ],
            "Resource": ["*"]
        }
    ]
}

policy = iot.create_policy(
    policyName="iot_greengrass_workshop_policy",
    policyDocument=json.dumps(core_policy_doc)
)

### Attach the policy to the principal

In [ ]:
iot.attach_principal_policy(
    policyName=policy['policyName'],
    principal=bp_keys_cert['certificateArn']
)

iot.attach_principal_policy(
    policyName=policy['policyName'],
    principal=hr_keys_cert['certificateArn']
)

iot.attach_principal_policy(
    policyName=policy['policyName'],
    principal=ht_keys_cert['certificateArn']
)

### Configure Greengrass

To make setup more simplified there is a helper library available, but to understand the moving parts we will make the individual calls. The library source can be found at [Greengrass Group Setup](https://github.com/awslabs/aws-greengrass-group-setup)

In [ ]:
group = gg.create_group(Name="health_tracker")

### Create the core definition in to display in the AWS Console

In [ ]:
core_definition = gg.create_core_definition(
    Name="{0}_core_def".format(group['Name']),
    InitialVersion= {
        'Cores': [
            {
                'Id': ht_group_thing['thingName'],
                'CertificateArn': bp_keys_cert['certificateArn'],
                'SyncShadow': False, # Up to you, True|False
                'ThingArn': ht_group_thing['thingArn']
            }
        ]
    }
)

### Crete the initial version of the group and device

In [ ]:
group_ver = gg.create_group_version(
    GroupId=group['Id'],
    CoreDefinitionVersionArn=core_definition['LatestVersionArn']
)

In [ ]:
print ("Group: https://.console.aws.amazon.com/iot/home?region={}#/greengrass/groups/{}".format(region, group['Id']))
print ("BP thing: https://{0}.console.aws.amazon.com/iot/home?{0}#/thing/igw_bp_sensor".format(region))
print ("HR thing: https://{0}.console.aws.amazon.com/iot/home?{0}#/thing/igw_hr_sensor".format(region))
print ("BP Certificate and policy: https://{0}.console.aws.amazon.com/iot/home?region={0}#/certificate/{1}".format(region, bp_keys_cert['certificateId']))
print ("HR Certificate and policy: https://{0}.console.aws.amazon.com/iot/home?region={0}#/certificate/{1}".format(region, hr_keys_cert['certificateId']))


In [ ]:
state = {
    'group': group,
    'bp_thing': bp_thing,
    'hr_thing': hr_thing,
    'bp_keys_cert': bp_keys_cert,
    'hr_keys_cert': hr_keys_cert,
    'group_ver': group_ver,
    'core_definition': core_definition,
    'policy': policy
}

In [ ]:
with open('./state.json', 'w') as f:
    json.dump(state, f, indent=4)
    
state

In [ ]:
response = iot.describe_endpoint()
tempIoTHost = response['endpointAddress']
tempGGHost = 'greengrass.iot.' + region + '.amazonaws.com'

print(tempIoTHost)
print(tempGGHost)

In [ ]:
with open('./iot/iot-bp-pem-crt', 'w') as f:
    f.write(bp_keys_cert['certificatePem'])

with open('./iot/iot-bp-pem-key', 'w') as f:
    f.write(bp_keys_cert['keyPair']['PrivateKey'])

config = {
    "coreThing": {
        "caPath": "root.ca.pem",
        "certPath": "iot-bp-pem-crt",
        "keyPath": "iot-bp-pem-key",
        "thingArn": bp_thing['thingArn'],
        "iotHost": tempIoTHost,
        "ggHost": tempGGHost,
        "keepAlive" : 600
    },
    "runtime": {
        "cgroup": {
            "useSystemd": "yes"
        }
    },
    "managedRespawn": False
}
with open('./iot/bp_config.json', 'w') as f:
    json.dump(config, f, indent=4)

In [ ]:
with open('./iot/iot-hr-pem-crt', 'w') as f:
    f.write(hr_keys_cert['certificatePem'])

with open('./iot/iot-hr-pem-key', 'w') as f:
    f.write(hr_keys_cert['keyPair']['PrivateKey'])

config = {
    "coreThing": {
        "caPath": "root.ca.pem",
        "certPath": "iot-hr-pem-crt",
        "keyPath": "iot-hr-pem-key",
        "thingArn": hr_thing['thingArn'],
        "iotHost": tempIoTHost,
        "ggHost": tempGGHost,
        "keepAlive" : 600
    },
    "runtime": {
        "cgroup": {
            "useSystemd": "yes"
        }
    },
    "managedRespawn": False
}
with open('./iot/hr_config.json', 'w') as f:
    json.dump(config, f, indent=4)

## Cleanup

In [ ]:
ec2_client.terminate_instances(InstanceIds=[instance_id])

waiter=ec2_client.get_waiter('instance_terminated')
waiter.wait(InstanceIds=[instance_id])

In [3]:
workshop.vpc_cleanup(vpc_id)

Removing VPC (vpc-0b15bdb93cec1d509) from AWS


In [ ]:
response = ec2_client.delete_key_pair(
    KeyName=project_name,
)